# Work Summary

In [347]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from time import time
import math
import matplotlib.pyplot as plt
% matplotlib inline

plt.style.use("ggplot")

In [348]:
df = pd.read_csv('../data/df_rating.csv')

In [349]:
df.head()

,uid,song_id,rating
0,10199495,22820742,2.0
1,104213634,6096002,1.0
2,104992781,127709,4.0
3,104992781,708290,4.0
4,10607827,22872742,4.0


## 1. Clean data and create utility matrix

#### Select relevant columns in the original dataframe

In [350]:
# Get song_id, uid, rating for recommender

df_recommender = df[['song_id', 'uid', 'rating']]
df_recommender.shape

(2539015, 3)

#### To reduce compute time, we apply downsample on song_id level

In [351]:
# user_id list
df_user_list = np.array(df_recommender['uid'].drop_duplicates())

In [352]:
print("total number of users:",len(df_user_list))

total number of users: 56694


In [353]:
# downsample ids
np.random.seed = 1
down_sample_ratio = 0.05
id_subset = pd.DataFrame(df_user_list[np.random.random(df_user_list.shape)<down_sample_ratio])
id_subset.columns = ['uid']

In [354]:
print("total number of users after down sample:",len(id_subset))

total number of users after down sample: 2908


In [355]:
# retain only random seleted samples
df_recommender_down_sample = pd.merge(id_subset, df_recommender, on = 'uid', how = 'left')
df_recommender_down_sample.shape

(128897, 3)

#### There are many users that haven't play many songs, exclude these users from the utility matrix, retain only users play more than five songs.
#### For the removed or new users, we can recommend popular songs at first.

In [356]:
# filter 'uid' play more than 5 songs
df_user = df_recommender_down_sample['uid'].value_counts()
df_user = df_user[df_user >= 5]

In [357]:
df_user.count(), df_user.sum()

(2162, 127419)

In [358]:
# filter users play more than 5 songs and set user_id as index
df_recommender_cleaned = df_recommender_down_sample.set_index('uid').ix[df_user.index].reset_index()
df_recommender_cleaned.head(2)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,uid,song_id,rating
0,169031835,1108569,5.0
1,169031835,169744,3.0


In [359]:
df_recommender_cleaned.shape

(127419, 3)

##### extract the unique list of user_id and song_id

In [360]:
user_id_list = pd.DataFrame(df_recommender_cleaned['uid'].unique())
user_id_list = user_id_list.reset_index()
# Rename the columns 
user_id_list.columns = ['user_number', 'uid']
user_id_list.head(2)

,user_number,uid
0,0,169031835
1,1,168999447


In [361]:
song_id_list = pd.DataFrame(df_recommender_cleaned['song_id'].unique())
song_id_list = song_id_list.reset_index()
# Rename the columns 
song_id_list.columns = ['song_number', 'song_id']
song_id_list.head(2)

,song_number,song_id
0,0,1108569
1,1,169744


In [362]:
# merge "df_recommender_cleaned", "user_id_list" and "song_id_list"
df_recommender_cleaned = pd.merge(df_recommender_cleaned, user_id_list, on = 'uid', how = 'left')
df_recommender_cleaned = pd.merge(df_recommender_cleaned, song_id_list, on = 'song_id', how = 'left')

df_recommender_cleaned.head(2)

,uid,song_id,rating,user_number,song_number
0,169031835,1108569,5.0,0,0
1,169031835,169744,3.0,0,1


#### Create utility matrix from records

##### convert to sparse matrix using scipy.sparse.lil_matrix

In [363]:
highest_user_id = df_recommender_cleaned.user_number.max()
highest_song_id = df_recommender_cleaned.song_number.max()
ratings_mat = sparse.lil_matrix((highest_user_id + 1, highest_song_id + 1))
ratings_mat

<2162x51208 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in LInked List format>

In [364]:
# filter in utility_mat with rating
for _, row in df_recommender_cleaned.iterrows():
    ratings_mat[row.user_number, row.song_number] = row.rating

In [365]:
ratings_mat

<2162x51208 sparse matrix of type '<class 'numpy.float64'>'
	with 127419 stored elements in LInked List format>

In [366]:
utility_mat = ratings_mat

## 2. Build Recommenders and insights
### 2.1 Popularity-based recommender

### For every new user or user played less than five songs, we build a Popularity-based recommender to recommend popular songs at first

In [388]:
# count by song_id and rank to find the mose popular songs
n = 10
Popularity_based_top_10 = df['song_id'].value_counts()[:n].tolist()
Popularity_based_top_100 = df['song_id'].value_counts()[:(n * 10)].tolist()

In [389]:
print('Popularity-based recommender recommends top ' + str(n) +' songs:', '\n', str(Popularity_based_top_10))

Popularity-based recommender recommends top 10 songs: 
 [10375, 9320, 6496, 6070, 5338, 5248, 4671, 4474, 4122, 3782]


### 2.2 Neighborhood-based Approach Collaborative Filtering Recommender
### 2.2.1 Item-Item similarity recommender 

### For user played more than five songs, we try Neighborhood-based approach to build an Item-Item similarity recommender here, given a user_id and recommend 10 songs which have the largest similarities with songs the user had played before.

#### Calculate item-item similarity matrix

In [369]:
# Item-Item Similarity Matrix
item_sim_mat = cosine_similarity(utility_mat.T)

#### Calculate neighborhood

In [370]:
least_to_most_sim_indexes = np.argsort(item_sim_mat, axis=1)

# Neighborhoods
neighborhood_size = 75
neighborhoods = least_to_most_sim_indexes[:, -neighborhood_size:]

In [371]:
neighborhoods.shape

(51208, 75)

#### Make rating prediction on a user

In [372]:
# Let's pick a lucky user
user_id = 100

In [373]:
n_users = utility_mat.shape[0]
n_items = utility_mat.shape[1]

start_time = time()
items_rated_by_this_user = ratings_mat[user_id].nonzero()[1]
# Just initializing so we have somewhere to put rating preds
out = np.zeros(n_items)
for item_to_rate in range(n_items):
    relevant_items = np.intersect1d(neighborhoods[item_to_rate],
                                    items_rated_by_this_user,
                                    assume_unique=True)  # assume_unique speeds up intersection op
    out[item_to_rate] = ratings_mat[user_id, relevant_items] * \
        item_sim_mat[item_to_rate, relevant_items] / \
        item_sim_mat[item_to_rate, relevant_items].sum()


pred_ratings = np.nan_to_num(out)
print(pred_ratings)
print("Execution time: %f seconds" % (time()-start_time))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


[0.         1.80817158 0.         ... 0.         0.         0.        ]
Execution time: 31.875315 seconds


In [374]:
pred_ratings.shape

(51208,)

#### Try to get final recommendations for a user: user_number = 100

In [375]:
# Recommend n songs
n = 10

# Get item indexes sorted by predicted rating
item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))[::-1]

# Find items that have been rated by user
items_rated_by_this_user = ratings_mat[user_id].nonzero()[1]

# We want to exclude the items that have been rated by user
unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                if item not in items_rated_by_this_user]

Item_Item_top_10 = unrated_items_by_pred_rating[:n]
Item_Item_top_100 = unrated_items_by_pred_rating[:(n * 10)]
print('Item-Item similarity recommender recommends top ' + str(n) +' songs:', '\n', str(Item_Item_top_10))

Item-Item similarity recommender recommends top 10 songs: 
 [42746, 45983, 38553, 38554, 38555, 38556, 38557, 45981, 45982, 45984]


In [376]:
### check errors
# truth
ratings_true = ratings_mat[user_id, items_rated_by_this_user].todense()
# prediction
ratings_pred = pred_ratings[items_rated_by_this_user]
# print(list(zip(np.array(ratings_true).squeeze(),ratings_pred)))
err_one_user = ratings_true-ratings_pred
# print(err_one_user)
print("average_abs_err:", abs(err_one_user).mean())

average_abs_err: 0.9242829479252381


### Next we will try Matrix Factorization approach to build recommender, because matrix factorization models always perform better than neighborhood models in collaborative filtering. The reason as below:

### 2.3 Matrix Factorization Approach Collaborative Filtering Recommender

### Here we will compare two Matrix Factorization approaches: sklearn NMF and sklearn TruncatedSVD.

### 2.3.1 NMF

In [377]:
# train model

from sklearn.decomposition import NMF

def fit_nmf(M,k):
    nmf = NMF(n_components=k)
    nmf.fit(M)
    W = nmf.transform(M);
    H = nmf.components_;
    err = nmf.reconstruction_err_
    return W,H,err

# decompose
W,H,err = fit_nmf(ratings_mat,700)
print(err)
print(W.shape,H.shape)

516.2219042522587
(2162, 700) (700, 51208)


In [378]:
# calculate model performance

# reconstruct
ratings_mat_fitted = W.dot(H)
errs = np.array((ratings_mat-ratings_mat_fitted).flatten()).squeeze()
mask = np.array((ratings_mat.todense()).flatten()).squeeze()>0

rmse = math.sqrt(np.mean(errs[mask]**2))
average_abs_err = abs(errs[mask]).mean()
print("RMSE: ", rmse)
print("average_abs_err: ", average_abs_err)

RMSE:  1.3431440657800777
average_abs_err:  0.6158359615821906


### The RMSE of NMF is 1.3431, and the average absolute error is 0.6158, the performance is acceptable.
#### Next we will try to get recommendations for a user: user_number = 100

In [379]:
# get recommendations for one user
user_id = 100
n = 10

pred_ratings = ratings_mat_fitted[user_id,:]
item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))[::-1]

items_rated_by_this_user = ratings_mat[user_id].nonzero()[1]

unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                if item not in items_rated_by_this_user]

NMF_top_10 = unrated_items_by_pred_rating[:n]
NMF_top_100 = unrated_items_by_pred_rating[:(n * 10)]
print('NMF recommender recommends top ' + str(n) +' songs:', '\n', str(NMF_top_10))

NMF recommender recommends top 10 songs: 
 [45979, 45977, 45978, 10569, 26479, 9938, 23038, 9869, 9880, 23062]


In [380]:
### check errors
# truth
ratings_true = ratings_mat[user_id, items_rated_by_this_user].todense()
# prediction
ratings_pred = pred_ratings[items_rated_by_this_user]
# print(list(zip(np.array(ratings_true).squeeze(),ratings_pred)))
err_one_user = ratings_true-ratings_pred
# print(err_one_user)
print("average_abs_err:", abs(err_one_user).mean())

average_abs_err: 0.022738391411277273


### The same as what we discussed above, the average absolute error of NMF for this specific user is better than 0.92 of Item-Item similarity recommender.
#### Next we will try TruncatedSVD to verify whether it performs better than NMF.

### 2.3.2 TruncatedSVD

In [381]:
# train model
from sklearn.decomposition import TruncatedSVD

def fit_uvd(M,k):
    # use TruncatedSVD to realize UVD
    svd = TruncatedSVD(n_components=k, n_iter=7, random_state=0)
    svd.fit(M)

    V = svd.components_
    U = svd.transform(M) # effectively, it's doing: U = M.dot(V.T)
    # we can ignore svd.singular_values_ for our purpose
    
    return U,V, svd

# decompose
U,V,svd = fit_uvd(ratings_mat,700)

print(U.shape,V.shape)

(2162, 700) (700, 51208)


In [382]:
# calculate model performance

# reconstruct
ratings_mat_fitted = U.dot(V) # U*V

# recall: U = M.dot(V.T), then this is M.dot(V.T).dot(V)
# original M is transformed to new space, then transformed back
# this is another way to understand it!

# calculate errs
errs = np.array((ratings_mat-ratings_mat_fitted).flatten()).squeeze()
mask = np.array((ratings_mat.todense()).flatten()).squeeze()>0

rmse = math.sqrt(np.mean(errs[mask]**2))
average_abs_err = abs(errs[mask]).mean()
print("RMSE: ", rmse)
print("average_abs_err: ", average_abs_err)

RMSE:  1.1701637826723559
average_abs_err:  0.5621628332473892


### The RMSE of TruncatedSVD is 1.1702 and the average absolute error is 0.56, which are better than scores of NMF(1.3431 and 0.6158). 
#### TruncatedSVD performs better because it has larger degree of freedom than NMF, to be specific, NMF is a specialization of TruncatedSVD, all values of V, W, and H in NMF must be non-negative.


#### Next we will try to get recommendations for a user: user_number = 100

In [383]:
# get recommendations for one user
user_id = 100
n = 10

pred_ratings = ratings_mat_fitted[user_id,:]
item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))[::-1]

items_rated_by_this_user = ratings_mat[user_id].nonzero()[1]

unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                if item not in items_rated_by_this_user]

TruncatedSVD_top_10 = unrated_items_by_pred_rating[:n]
TruncatedSVD_top_100 = unrated_items_by_pred_rating[:(n * 10)]
print('TruncatedSVD recommender recommends top ' + str(n) +' songs:', '\n', str(TruncatedSVD_top_10))

TruncatedSVD recommender recommends top 10 songs: 
 [258, 7341, 7512, 18055, 28364, 658, 13202, 45719, 48377, 48378]


In [384]:
### check errors
# truth
ratings_true = ratings_mat[user_id, items_rated_by_this_user].todense()
# prediction
ratings_pred = pred_ratings[items_rated_by_this_user]
# print(list(zip(np.array(ratings_true).squeeze(),ratings_pred)))
err_one_user = ratings_true-ratings_pred
# print(err_one_user)
print("average_abs_err:", abs(err_one_user).mean())

average_abs_err: 0.04404674113511854


### The average absolute error of TruncatedSVD is 0.044, which is very close to 0.023 of NMF.

### 2.4. Specific recommendation results comparison and insights

### Next we compare the recommendation results for 'user with user_number = 100' generated by the four models above.

#### We generate the overlap tabel of the top_10 results given by the four models for 'user with user_number = 100'

In [391]:
top10_overlap = pd.DataFrame([len(np.intersect1d(Popularity_based_top_10, Item_Item_top_10)),
                                   len(np.intersect1d(Popularity_based_top_10, TruncatedSVD_top_10)),
                                   len(np.intersect1d(Popularity_based_top_10, NMF_top_10)),
                                   len(np.intersect1d(Item_Item_top_10, NMF_top_10)),
                                   len(np.intersect1d(Item_Item_top_10, TruncatedSVD_top_10)),
                                   len(np.intersect1d(NMF_top_10, TruncatedSVD_top_10)),
                                   len(np.intersect1d(np.intersect1d(np.intersect1d(Popularity_based_top_10, Item_Item_top_10), NMF_top_10), TruncatedSVD_top_10))],
                                 index = ['Popularity-based_with_Item-Item',
                                           'Popularity-based_with_NMF',
                                           'Popularity-based_with_TruncatedSVD',
                                           'Item-Item_with_NMF',
                                           'Item-Item_with_TruncatedSVD',
                                           'NMF_with_TruncatedSVD',
                                           'Popularity-based_with_Item-Item_with_NMF_with_TruncatedSVD'],
                                 columns = ['number_of_same_recommended_songs_in_top_10'])
print('The overlap of the top 10 recommendation generated by these four models') 
top10_overlap

The overlap of the top 10 recommendation generated by these four models


,number_of_same_recommended_songs_in_top_10
Popularity-based_with_Item-Item,0
Popularity-based_with_NMF,0
Popularity-based_with_TruncatedSVD,0
Item-Item_with_NMF,0
Item-Item_with_TruncatedSVD,0
NMF_with_TruncatedSVD,0
Popularity-based_with_Item-Item_with_NMF_with_TruncatedSVD,0


#### We generate the overlap tabel of the top_100 results given by the four models for 'user with user_number = 100'

In [392]:
top100_overlap = pd.DataFrame([len(np.intersect1d(Popularity_based_top_100, Item_Item_top_100)),
                                   len(np.intersect1d(Popularity_based_top_100, TruncatedSVD_top_100)),
                                   len(np.intersect1d(Popularity_based_top_100, NMF_top_100)),
                                   len(np.intersect1d(Item_Item_top_100, NMF_top_100)),
                                   len(np.intersect1d(Item_Item_top_100, TruncatedSVD_top_100)),
                                   len(np.intersect1d(NMF_top_100, TruncatedSVD_top_100)),
                                   len(np.intersect1d(np.intersect1d(np.intersect1d(Popularity_based_top_100, Item_Item_top_100), NMF_top_100), TruncatedSVD_top_100))],
                                 index = ['Popularity-based_with_Item-Item',
                                           'Popularity-based_with_NMF',
                                           'Popularity-based_with_TruncatedSVD',
                                           'Item-Item_with_NMF',
                                           'Item-Item_with_TruncatedSVD',
                                           'NMF_with_TruncatedSVD',
                                           'Popularity-based_with_Item-Item_with_NMF_with_TruncatedSVD'],
                                 columns = ['number_of_same_recommended_songs_in_top_100'])
print('The overlap of the top 100 recommendation generated by these four models') 
top100_overlap

The overlap of the top 100 recommendation generated by these four models


,number_of_same_recommended_songs_in_top_100
Popularity-based_with_Item-Item,0
Popularity-based_with_NMF,0
Popularity-based_with_TruncatedSVD,0
Item-Item_with_NMF,0
Item-Item_with_TruncatedSVD,0
NMF_with_TruncatedSVD,30
Popularity-based_with_Item-Item_with_NMF_with_TruncatedSVD,0


### From the intersets tables above, we notice that: 

### Conlcusion: